# "Progressive Neural Networks" paper implementation - https://arxiv.org/pdf/1606.04671.pdf

In [ ]:
import os
import shutil
from pprint import pformat

import torch
import torch.multiprocessing as mp

from optimizer import GlobalAdam
from params import Parameters
from testing import test
from training import train
from model import PNN
from environment import create_env

try:
    mp.set_start_method('spawn')
except RuntimeError:
    pass
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

opt = Parameters()

if opt.log_path.exists() and opt.log_path.is_dir():
    shutil.rmtree(opt.log_path)
opt.log_path.mkdir(parents=True)
if not opt.save_path.is_dir():
    opt.save_path.mkdir(parents=True)

allenvs = create_env(opt)
current = mp.Value('i', 0)
gmodel = PNN(allenvs, current)

gmodel.share_memory()
if opt.load:
    file = opt.save_path / "pnn"
    if file.exists() and file.is_file():
        gmodel.load_state_dict(torch.load(file))

optimizer = GlobalAdam(gmodel.parameters(), lr=opt.lr)

processes = []

for pid in range(opt.nprocesses):
    process = mp.Process(target=train,
                          args=(pid, opt, current, gmodel, optimizer))
    process.start()
    processes.append(process)

process = mp.Process(target=test, args=(opt.nprocesses, opt, gmodel))
process.start()
processes.append(process)

for process in processes:
    process.join()